<h1 style="text-align: center; color: #086A87;">Web Scrapping Python Script</h1>
<h3 style="text-align: center; color: #086A87;">By: McKenzie Kulseth</h3><br><br>
<h4 style="color: #086A87;margin:0;">Purpose of this code:</h4>
<p style="color: #086A87; margin:0;">This code will parse a webpage for data and extract that data. 
    <br><br><em>NOTE: One should orgainze the csv files being save into a directory tree to better organize the files.</p>
<p style="color: #086A87;"><br><strong>1.) Import Libraries</p>


In [ ]:
import requests as req
import lxml
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from datetime import *

<p style="color: #086A87;margin: 0;"> <br> <strong>2.) Get the station ids for the sounding stations from a website</strong>
<ul style="color: #086A87;margin: 0;"><li> Initialize list to ensure no station id is repeated in the list
    <li> Loop through the website in the list of websites
    <li> Use BeautifulSoup to parse the webpage for the station ids without the 'k' in front front
    <li> Append those ids to the list of station ids</ul>
</p>

In [ ]:
websites = ["http://www.weathergraphics.com/identifiers/master-station.dat"]
station_ids = []
for ws in websites:
    source = req.get(ws).text
    soup = BeautifulSoup(source,'lxml')
    bod = soup.find('body')
    text1 = bod.p.text
    mytext1 = text1.split(":")
    data1 = mytext1[-1].replace("_","")
    data =  data1.replace("\r","").split("\n")
    headers = data[0].split(" ")
    for row in data[1:]:
        try:
            if row[0] == "K" and "-9999" not in row:
                station_ids.append(row[1:4])
        except:
            continue
print("Done!, There are "+str(len(station_ids))+" stations in North America.")

<p style="color: #086A87;margin: 0;"> <br><strong>3.) Creating lists to hold the dates to be looped through to find sounding data.</strong>
<br> <ul style="color: #086A87;margin: 0;"><li> Get today's date to get the ending year for sounding data to search for
    <li> Create lists of years, months, and days to search for sounding data
</ul></p>

In [ ]:
today = date.today()
this_year = int(today.strftime("%Y"))
years = np.arange(1973,this_year+1)
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

days = ['01','02','03','04','05','06','07','08','09','10',
          '11','12','13','14','15','16','17','18','19','20',
          '21','22','23','24','25','26','27','28','29','30','31']
print("Done creating years, months, and days to loop through for getting data.")

<p style="color: #086A87;"><br><strong>4.) Loop through those dates and station ids to create a list of website to be parsed for sounding data</p>

In [ ]:
web_sites = []
for y in years:
    for m in months:
        for d in days:
            for ids in station_ids:
                web_sites.append("http://weather.uwyo.edu/cgi-bin/sounding?region=naconf&TYPE=TEXT%3ALIST&YEAR="
                                +str(y)+"&MONTH="+str(m)+"&FROM="
                                +str(d)+"00"+"&TO="+str(d)+"12"+"&STNM="+ids[1:])
print("Done. There are "+str(len(web_sites))+" to sift through.")

<p style="color: #086A87;margin: 0;"><br><strong> 5.) Get sounding data from websites</strong>
<ul style="color: #086A87;margin: 0;"><li> Loop through websites
    <li> Store sounding data in dictionary
    <li> Create a panadas dataframe to store the dictionary
    <li> Save the dataframe as a csv file
</ul>
</p>

In [ ]:
counting_ids_w_no_data = 0; websites_with_no_sounding_data = []
websites_no_data = {}
for ws in web_sites[0:3]:
    sounding_data = {"PRES[hPa]":[], "HGHT[m]":[], "TEMP[C]":[], "DWPT[C]":[], "RELH[%]":[],
                     "MIXR[g/kg]":[], "DRCT[deg]":[], "SKNT[knot]":[], "THTA[K]":[],
                     "THTE[K]":[], "THTV[K]":[]}
    file_name = ''
    mydf = pd.DataFrame()
    headers = []
    units = []
    try:
        source = req.get(ws).text
        soup = BeautifulSoup(source,"html.parser")
        data = soup.find("pre").text
        data1 = data.split("\n")
        if ws[-3] != "=":
            file_name = "K"+ws[-3:]+"_"+ws[-40:-36]+"_"+ws[-29:-27]+"_"+ws[-13:-11]+"_sounding_data.csv"
        else:
            file_name = "K"+ws[-2:]+"_"+ws[-39:-35]+"_"+ws[-28:-26]+"_"+ws[-12:-10]+"_sounding_data.csv"
        headers = data1[2]
        units = data1[3]
        if headers != []:
            for row in data1[5:]:
                d1 = row[16:21].replace(" ",""); d2 = row[23:28].replace(" ","")
                d3 = row[33:35].replace(" ",""); d4 = row[38:42].replace(" ","")
                d5 = row[47:50].replace(" ",""); d6 = row[50:57].replace(" ","")
                d7 = row[59:64].replace(" ",""); d8 = row[66:71].replace(" ","")
                d9 = row[73:].replace(" ",""); d10 = row[1:7].replace(" ","")
                d11 = row[9:14].replace(" ","")
                if len(d1) != 0:
                    sounding_data["TEMP[C]"].append(float(d1))
                elif len(d1) == 0:
                    sounding_data["TEMP[C]"].append(np.nan)
                if len(d2) != 0:
                    sounding_data["DWPT[C]"].append(float(d2))
                elif len(d2) == 0:
                    sounding_data["DWPT[C]"].append(np.nan)
                if len(d3) != 0:
                    sounding_data["RELH[%]"].append(float(d3))
                elif len(d3) == 0:
                    sounding_data["RELH[%]"].append(np.nan)
                if len(d4) != 0:
                    sounding_data["MIXR[g/kg]"].append(float(d4))
                elif len(d4) == 0:
                    sounding_data["MIXR[g/kg]"].append(np.nan)
                if len(d5) != 0:
                    sounding_data["DRCT[deg]"].append(float(d5))
                elif len(d5) == 0:
                    sounding_data["DRCT[deg]"].append(np.nan)
                if len(d6) != 0:
                    sounding_data["SKNT[knot]"].append(float(d6))
                elif len(d6) == 0:
                    sounding_data["SKNT[knot]"].append(np.nan)
                if len(d7) != 0:
                    sounding_data["THTA[K]"].append(float(d7))
                elif len(d7) == 0:
                    sounding_data["THTA[K]"].append(np.nan)
                if len(d8) != 0:
                    sounding_data["THTE[K]"].append(float(d8))
                elif len(d8) == 0:
                    sounding_data["THTE[K]"].append(np.nan)
                if len(d9) != 0:
                    sounding_data["THTV[K]"].append(float(d9))
                elif len(d9) == 0:
                    sounding_data["THTV[K]"].append(np.nan)
                if len(d10) != 0:
                    sounding_data["PRES[hPa]"].append(float(d10))
                elif len(d10) == 0:
                    sounding_data["PRES[hPa]"].append(np.nan)
                if len(d11) != 0:
                    sounding_data["HGHT[m]"].append(float(d11))
                elif len(d11) == 0:
                    sounding_data["HGHT[m]"].append(np.nan)
        mydf = pd.DataFrame(sounding_data)
        mydf.to_csv("data/"+file_name)
    except:
        if ws[-3] != "=":
            if ws[-3] not in websites_with_no_sounding_data:
                websites_with_no_sounding_data.append(ws[-3:])
                websites_no_data.update({"K"+ws[-3:]:[]})                
            websites_no_data["K"+ws[-3:]].append(ws[-29:-27]+"/"+ws[-13:-11]+"/"+ws[-40:-36])
            file_name = "K"+ws[-3:]+"_"+ws[-40:-36]+"_"+ws[-29:-27]+"_"+ws[-13:-11]+"_sounding_data.csv"
        else:
            if ws[-2] not in websites_with_no_sounding_data:
                websites_with_no_sounding_data.append(ws[-2:])
                websites_no_data.update({"K"+ws[-2:]:[]})                
            websites_no_data["K"+ws[-2:]].append(ws[-28:-26]+"/"+ws[-12:-10]+"/"+ws[-39:-35])
            file_name = "K"+ws[-2:]+"_"+ws[-39:-35]+"_"+ws[-28:-26]+"_"+ws[-12:-10]+"_sounding_data.csv"
        counting_ids_w_no_data += 1

print(str(counting_ids_w_no_data)+" of the "+str(len(web_sites))+" websites had no sounding information.")

<p style="color: #086A87;"><strong>6.) View the station ids and the dates they had know sounding data</strong></p>

In [ ]:
print("Station_ID\tDates")
for key in websites_no_data.keys():
    print(key+"\t\t",end="")
    for d in websites_no_data[key]:
        if d != websites_no_data[key][-1]:
            print(d,end=", ")
        else:
            print(d+".")
